In [1]:
# --- 配置：把下面这行改成你的作业文件夹路径 ---
FOLDER = r"./第8次作业"   # 例如：r"D:\作业\第3次" 或 "/home/you/作业/第3次"
OUTPUT_XLSX = "第8次作业提交统计情况.xlsx"  # 输出文件名
INCLUDE_DIRS = False         # True 则也尝试把子文件夹名当作作业名解析
RECURSIVE = False            # True 则递归所有子目录下的文件/文件夹

# --- 代码开始：直接运行即可 ---
import re
from pathlib import Path
from collections import OrderedDict

import pandas as pd  # 需要：pip install pandas openpyxl

# 允许的分隔符：空格/_/-
SEP = r"[ _-]?"

# k 可以是阿拉伯数字或常见中文数字（包含“十百千两〇”等）
CN_NUM = "零一二三四五六七八九十百千两〇"
K_PATTERN = rf"[{CN_NUM}0-9]+"

# 匹配模式：学号(8位) + [分隔符可有可无] + 姓名(最短匹配到“第”前) + [分隔符可有可无] + 第k次作业
PATTERN = re.compile(
    # rf"^\s*(?P<id>\d{{8}}){SEP}(?P<name>.+?){SEP}第(?P<k>{K_PATTERN})次作业",
    rf"^\s*(?P<id>\d{{8}}){SEP}(?P<name>.+?){SEP}第.*",
    re.IGNORECASE,
)

def parse_filename(stem: str):
    """
    输入不含扩展名的文件名（stem），返回 (id, name); 不匹配则抛出 ValueError
    """
    m = PATTERN.match(stem)
    if not m:
        raise ValueError("不匹配‘学号+姓名+第k次作业’格式")
    sid = m.group("id")
    name = m.group("name").strip()
    if not sid.isdigit() or len(sid) != 8:
        raise ValueError("学号不是8位数字")
    if not name:
        raise ValueError("姓名为空")
    return sid, name

def collect_paths(root: Path, include_dirs=False, recursive=False):
    if recursive:
        it = root.rglob("*")
    else:
        it = root.iterdir()
    for p in it:
        if p.is_file():
            yield p, p.stem  # 文件：用不含扩展名的 stem
        elif include_dirs and p.is_dir():
            yield p, p.name  # 目录：直接用目录名

def build_lists(folder, include_dirs=False, recursive=False):
    root = Path(folder)
    if not root.exists() or not root.is_dir():
        raise SystemExit(f"路径无效或不是文件夹：{root}")

    entries = []
    invalid = []

    for p, stem in collect_paths(root, include_dirs=include_dirs, recursive=recursive):
        # 跳过空名/隐藏项的常见情况（可选）
        if not stem.strip():
            invalid.append({"文件名": p.name, "原因": "空文件名/目录名"})
            continue
        try:
            sid, name = parse_filename(stem)
            entries.append({"学号": sid, "姓名": name, "源文件名": p.name})
        except Exception as e:
            invalid.append({"文件名": p.name, "原因": str(e)})

    # 去重：按学号唯一；若同一学号对应多个姓名，记入“不合法文件”
    id_to_name = OrderedDict()
    for row in entries:
        sid = row["学号"]; name = row["姓名"]
        if sid not in id_to_name:
            id_to_name[sid] = name
        else:
            if id_to_name[sid] != name:
                invalid.append({
                    "文件名": row["源文件名"],
                    "原因": f"同一学号出现不同姓名：{id_to_name[sid]} / {name}"
                })

    valid_rows = [{"学号": sid, "姓名": name} for sid, name in id_to_name.items()]
    return valid_rows, invalid

def export_xlsx(valid_rows, invalid_rows, outpath="result.xlsx"):
    with pd.ExcelWriter(outpath, engine="openpyxl") as writer:
        pd.DataFrame(valid_rows).to_excel(writer, index=False, sheet_name="有效名单")
        # 确保“不合法文件”表至少有表头
        df_invalid = pd.DataFrame(invalid_rows) if invalid_rows else pd.DataFrame(columns=["文件名","原因"])
        df_invalid.to_excel(writer, index=False, sheet_name="不合法文件")
    return Path(outpath).resolve()

# 执行
valid, invalid = build_lists(FOLDER, include_dirs=INCLUDE_DIRS, recursive=RECURSIVE)
xlsx_path = export_xlsx(valid, invalid, OUTPUT_XLSX)

print(f"已导出：{xlsx_path}")
print(f"有效记录：{len(valid)} 条；不合法：{len(invalid)} 条")

# 在 notebook 中预览前几行（可选）
display(pd.DataFrame(valid).head())
display(pd.DataFrame(invalid).head())


已导出：D:\Jupyter\第7次作业提交统计情况.xlsx
有效记录：98 条；不合法：0 条


,学号,姓名
0,22962096,SUZUKI MUNENARI（铃木宗诚）
1,25308001,包天宝
2,25308002,宾振杰
3,25308003,蔡冰悦
4,25308004,蔡博燊


""
